# Assembler texte à partir d'une liste d'EAF

In [1]:
import codecs
import pandas as pd
from unidecode import unidecode
from lxml import etree as ET

### Lire les informations sur l'ordre d'assemblage des EAF

In [32]:
repSource="/Users/gilles/Downloads/"
repEAF=repSource+"Cosedi-FranceNouvelleZelande/"
fSync=repSource+"France-NouvelleZelande.xlsx"
repEAF=repSource+"Cosedi-FranceArgentine/"
fSync=repSource+"France-Argentine.xlsx"
fTexte=fSync.replace(".xlsx",".txt")
df=pd.read_excel(fSync)

In [33]:
# df.dropna(thresh=3)

In [34]:
df["prenom"]=df.dropna(thresh=3)[u"Prénom"].str.strip().str.upper().str.replace(" ","-").apply(unidecode)
df["nom"]=df.dropna(thresh=3)[u"Nom"].str.strip().str.upper().str.replace(" ","-").apply(unidecode)
df["fichier"]=df.prenom+"-"+df.nom+".eaf"

In [35]:
dfSync=df.dropna(thresh=3)[u"début fin fichier".split(" ")]

In [36]:
fichiers=dfSync.fichier.tolist()
fichiers

['JEANNE-ROBIN.eaf',
 'AXEL-MEUNIER.eaf',
 'CAMILLE-VILLECHALANNE.eaf',
 'MAEWENN-GAUDIN.eaf',
 'CAPUCINE-MERESSE-ETIENNE.eaf',
 'MAUREEN-VIARD.eaf',
 'LEELOU-DE-GIRARDI.eaf',
 'COLINE-GOURSAUD.eaf',
 'LAETITIA-CAPELO.eaf',
 'GAELLE-DUBROCA.eaf',
 'EVA-DELAHODDE.eaf',
 'LILIANA-CORREIA.eaf',
 'JULIE-BOATTO.eaf',
 'ESTELLE-VEAU.eaf',
 'MARIAH-GOTTER.eaf',
 'ELYSA-RAVET.eaf',
 'ALISSA-BESSETTES.eaf',
 'ELISE-CHAIGNE.eaf',
 'LAURINE-BIGLIONE.eaf',
 'GAELLE-LAMARQUE.eaf',
 'ELIANE-MEGARUS.eaf',
 'LEA-BOUSSARD.eaf',
 'LEO-MESQUITA.eaf',
 'MARIUS-RECULET.eaf',
 'CHLOE-VERHEYDE.eaf',
 'CHARLINE-LEMOINE.eaf',
 'VADIM-OLIVAN.eaf',
 'CLARA-LEGER.eaf',
 'ALICE-LOOF.eaf',
 'YELYZAVETA-SHADRINA.eaf',
 'ROMAIN-DUBOIS.eaf',
 'COLEEN-MOREAU.eaf',
 'NORIKO-ROBINSON.eaf',
 'MATISSE-ROGER.eaf']

### Lire les fichiers EAF

In [37]:
parser = ET.XMLParser(remove_blank_text=True)

In [38]:
def time2TRS(time):
    return unicode(float(time)/1000)


def texteEAF(xmlEAF):
    ts={}
    for timeOrder in xmlEAF.xpath("//TIME_ORDER/TIME_SLOT"):
        tsID=timeOrder.attrib["TIME_SLOT_ID"]
        tsTime=timeOrder.attrib["TIME_VALUE"]
        ts[tsID]=tsTime
    tiersTypes={}
    annotations={}
    tcTexte={}
    turnTextes=[]
    for tier in xmlEAF.xpath("//TIER"):
        tierID=tier.attrib["TIER_ID"]
        if tierID!="Whisper":
            tierType=tier.attrib["LINGUISTIC_TYPE_REF"]
            tiersTypes[tierID]=tierType
            for annotation in tier.xpath("ANNOTATION/ALIGNABLE_ANNOTATION"):
                aID=annotation.attrib["ANNOTATION_ID"]
                aTS1=annotation.attrib["TIME_SLOT_REF1"]
                aTS2=annotation.attrib["TIME_SLOT_REF2"]
                aBegin=time2TRS(ts[aTS1])
                aEnd=time2TRS(ts[aTS2])
                aValue=annotation.xpath("ANNOTATION_VALUE/text()")
                # print aValue
                if aValue!=["[bruit]"]:
                    turnTextes.append("\n".join(aValue))
                    turn=(tierID, aBegin, aEnd, aValue)
                    annotations[aID]=turn
                    tcTexte[float(aBegin)]="\n".join(aValue)
    texteEAF=[]
    for k in sorted(tcTexte):
        texteEAF.append(tcTexte[k])
    return texteEAF

In [39]:
texteMatch=[]
for fichier in fichiers[:]:
    # print fichier
    try:
        xmlFichier=ET.parse(repEAF+fichier,parser)
        texteMatch.extend(texteEAF(xmlFichier))
    except:
        print "pas de",fichier

pas de COLINE-GOURSAUD.eaf
pas de GAELLE-DUBROCA.eaf
pas de LILIANA-CORREIA.eaf
pas de ALISSA-BESSETTES.eaf
pas de MARIUS-RECULET.eaf
pas de CHLOE-VERHEYDE.eaf
pas de VADIM-OLIVAN.eaf
pas de MATISSE-ROGER.eaf


In [28]:
with codecs.open(fTexte,"w",encoding="utf8") as outFile:
    outFile.write("\n".join(texteMatch))

In [29]:
texteMatch[-50:]

['il y va X',
 u'il est o\xfa le XXX',
 'oh non non y a pas essai',
 'en avant',
 'non non y a pas essai',
 'en avant',
 'y a pas essai',
 'non',
 'alors',
 u"et si s'il me fait \xe7a \xe0 tout long il me fait mourir",
 'ah ah ah ah ah',
 u"si il est \xe0 la but mais qu'il applatit pas il va il va me faire mourir",
 "X l'essai",
 u"et c'\xe9tait beau",
 u'ouais, ouais il y a pas grand chose \xe0, il y a pas grand chose \xe0 dire dire le',
 u'fin tout ceux qui \xe9taient sur le terrain ils \xe9taient e',
 'en moins de XX de nous et e',
 u"et \xe7a s'est vu d\xe8s d\xe8s la premi\xe8re mi-temps quand on a, quand il y a eu des opportunit\xe9es de de revenir on a pas \xe9taient e",
 u"assez rigoureux pour donner la quantit\xe9 \xe0 notre jeux pour tout donner X de l'espoir ",
 u"on a perdu des ballons vraiment trop facilement et contre eux \xe9videmment c'est e",
 'e on le paie, on le paie de suite donc e',
 u'voil\xe0 \xe7a a \xe9t\xe9 e, \xe7a a \xe9t\xe9 un milieu de deuxi\xe8me mi temp